# Visualize Today's Traffic Accidents in the City of Austin
##### Miguel Novo Villar
##### November 11, 2022 (More to be added)

## Modules

In [2]:
#Import data science modules
import pandas as pd
import numpy as np
import matplotlib as plt

#Scraping
from sodapy import Socrata

#Time
from datetime import datetime

#Geospatial
import geopandas as gpd
import contextily as cx

## Retrieve and transform data

##### We need to get data from data.austintexas.gov and autenticate to load the entire dataset

In [3]:
MyAppToken = "MyAppTokenfrom_data.austintexas.gov"

client = Socrata("data.austintexas.gov", #website
                MyAppToken, #token from portal
                 username="mymail@gmail.com", #username
                 password="mypasswords") #password

#Retrieve from the desired repository
results = client.get("dx9v-zd7x", limit = 1000000000)

# # Convert to pandas DataFrame, timestamp and sort by date descending
results_df = pd.DataFrame.from_records(results)
results_df["published_date"] = pd.to_datetime(results_df.published_date)
results_df= results_df.sort_values(by="published_date", ascending= False)
results_df.shape

(298208, 9)

# Transform to geopandas
Only today's data

In [4]:
today = results_df[(results_df['published_date'] > str(pd.to_datetime(datetime.utcnow().date())))]

Create geodataframe

In [5]:
gdf = gpd.GeoDataFrame(today, 
                        geometry=gpd.points_from_xy(today.longitude, today.latitude),
                        crs ="EPSG:4326").drop(["location","latitude","longitude"],axis = 1)
gdf.head(2)

,traffic_report_id,published_date,issue_reported,address,traffic_report_status,traffic_report_status_date_time,geometry
298207,EBA90709ED04EB27E4A042727D7DD308E44389E4_16680...,2022-11-10 03:13:19+00:00,Crash Service,Rutland Dr & N Lamar Blvd,ARCHIVED,2022-11-10T03:25:03.000Z,POINT (-97.69600 30.36487)
298204,9D65C4A27A601E62A599F3059159476CD76933A4_16680...,2022-11-10 03:06:51+00:00,Stalled Vehicle,10800 S Mopac Expy Sb,ACTIVE,2022-11-10T03:10:02.000Z,POINT (-97.87988 30.18377)


Visualize

In [6]:
gdf.drop("published_date", axis =1).explore(column='issue_reported', legend=True, marker_kwds=dict(radius=5))